In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dscim.utils.plotting_utils import MajorSymLogLocator, symlogfmt
from matplotlib.ticker import SymmetricalLogLocator, FuncFormatter

In [ ]:
# sectors=['agriculture_clipped', 'mortality_clipped', 'energy_clipped', 'labor_clipped', 'AMEL_clipped', "CAMEL_clipped"]
sectors=['CAMEL_clipped']
kinds=['zeroes', 'p0.01', 'p99.99']
recipe='risk_aversion'
disc='euler_ramsey'
eta_rhos = {
    2.0 :0.0,
    # 1.016010255: 9.149608e-05,
    # 1.244459066: 0.00197263997,
    # 1.421158116: 0.00461878399,
    # 1.567899395: 0.00770271076,
}
log=True

In [ ]:
for sector in sectors:
    for eta,rho in eta_rhos.items():
        for kind in kinds:
            for discounting in ['discounted_damages', 'discount_factors', 'marginal_damages']:

                results = f"/mnt/CIL_integration/rff3_with_disc_factors/iter0-19/{sector}/2020/unmasked_None"
                rff_ids = pd.read_csv(
                    f"{results}/rff_ids/{recipe}_{disc}_eta{eta}_rho{rho}_{kind}_rff_ids.csv"
                ).set_index(['rff_sp', 'simulation']).index

                damages = xr.open_zarr(
                    f"{results}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_marginal_damages.zarr"
                ).sel(weitzman_parameter='0.5', drop=True).marginal_damages

                sccs = xr.open_dataset(
                    f"{results}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_sccs.nc4"
                ).sel(weitzman_parameter='0.5', fair_aggregation='uncollapsed', drop=True).uncollapsed_sccs.stack(
                    {'rff_sp-simulation' :['rff_sp', 'simulation']}
                ).sel({'rff_sp-simulation' : rff_ids}
                     ).rename('SCC').to_dataframe().reset_index()
                
                df = xr.open_zarr(
                            f"{results}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_discount_factors.zarr"
                        ).sel(weitzman_parameter='0.5', drop=True).discount_factor

                if discounting=='discounted_damages':
                    data = damages * df
                elif discounting=='discount_factors':
                    data = df
                elif discounting=='marginal_damages':
                    data = damages

                data = data.stack(
                    {'rff_sp-simulation' :['rff_sp', 'simulation']}
                ).sel(
                    {'rff_sp-simulation' : rff_ids}
                ).rename('data').to_dataframe().reset_index()

                data = data.merge(sccs, on=['rff_sp', 'simulation'])

                fig, ax = plt.subplots(1,1,figsize=(15,8))

                sns.lineplot(
                    data=data,
                     x='year',
                     y='data',
                     hue='SCC',
                     ax=ax,
                )

                if log==True:
                    ax.set_yscale('symlog')
                
                ax.yaxis.set_major_locator(MajorSymLogLocator())
                ax.yaxis.set_major_formatter(FuncFormatter(symlogfmt))

                plt.subplots_adjust(top=0.85)
                fig.suptitle(f"{discounting}, SCC subset={kind} \n {sector} {recipe} {disc}, eta={eta} rho={rho}")

                plt.savefig(
                    f'/mnt/CIL_integration/rff_diagnostics/v3/damages_SCC_subsets/{sector}_{recipe}_{disc}_{eta}_{rho}__sccs-{kind}_{discounting}_log-{log}.png', 
                    bbox_inches='tight', 
                    dpi=300
                )